In [2]:
import pandas as pd
import numpy as np

These are the two files that will contain all of the information. We will add to them. 

In [3]:
fin_aid = pd.read_csv("data_files/Current Designations Student Financial Aid.csv")
advanced_find = pd.read_csv("data_files/Current Designations Advanced Find View.csv")

These are the files we pull the FOAP information from

In [4]:
fund_codes = pd.read_csv("data_files/All Acct Code - Funds.csv")
org_codes = pd.read_csv("data_files/All Acct Code - Org.csv")
acct_codes = pd.read_csv("data_files/All Acct Code - Acct Codes.csv")
prog_codes = pd.read_csv("data_files/All Acct Code - Program Codes.csv")

In [9]:
def parse_match_foaps(in_primary_df, fund_codes, org_codes, acct_codes, prog_codes):
    
    
    in_primary_df.reset_index()
    in_primary_df['fund_code'] = ''
    in_primary_df['fund_name'] = ''
    in_primary_df['org_code'] = ''
    in_primary_df['org_name'] = ''
    in_primary_df['acct_code'] = ''
    in_primary_df['acct_name'] = ''
    in_primary_df['prog_code'] = ''
    in_primary_df['prog_name'] = ''
    in_primary_df['Credit Account'] = advanced_find['Credit Account'].replace(np.NaN, " ")
    
    
    
    for index, row in in_primary_df.iterrows():

        long_code = row['Credit Account']
        if len(long_code) > 2:
            long_code = row['Credit Account'].replace("^", "").split('|')

            fund_num = long_code[3]
            org_num = long_code[4]
            acct_num = long_code[5]
            prog_num = long_code[6]

            in_primary_df.at[index, 'fund_code'] = fund_num
            in_primary_df.at[index, 'org_code'] = org_num
            in_primary_df.at[index, 'acct_code'] = acct_num
            in_primary_df.at[index, 'prog_code'] = prog_num

            in_primary_df.at[index, 'fund_name'] = \
                tuple(fund_codes.loc[fund_codes['\'Fund Code\''] == fund_num, '\'Title\''])[-1]

            if len(org_num) > 0:
                in_primary_df.at[index, 'org_name'] = \
                    tuple(org_codes.loc[org_codes['\'Organization Code\''] == int(org_num), '\'Title\''])[-1]    

            if len(acct_num) > 0:
                in_primary_df.at[index, 'acct_name'] = \
                    tuple(acct_codes.loc[acct_codes['\'Account Code\''] == int(acct_num), '\'Title\''])[-1]

            if len(prog_num) > 0:
                in_primary_df.at[index, 'prog_name'] = \
                    tuple(prog_codes.loc[prog_codes['\'Program Code\''] == int(prog_num), '\'Title\''])[-1]        

        else:
            #print("\nNo code!")
            pass
    return in_primary_df


In [13]:
parse_match_foaps(advanced_find, fund_codes, org_codes, acct_codes, prog_codes)\
    .to_csv("output/Current Designations Advanced Find View With FOAPs.csv", index = False)
parse_match_foaps(fin_aid, fund_codes, org_codes, acct_codes, prog_codes)\
    .to_csv("output/Current Designations Student Financial Aid With Foaps.csv", index = False)